In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-17 11:55:06--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.2’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-17 11:55:06 (5.57 MB/s) - ‘course_lead_scoring.csv.2’ saved [80876/80876]



In [42]:
df = pd.read_csv("/kaggle/working/course_lead_scoring.csv")

In [43]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [44]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [45]:
categorical = ["lead_source", "industry", "employment_status", "location"]
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score", "converted"]

In [46]:
df[numerical] = df[numerical].fillna(0.0)

In [47]:
df[categorical] = df[categorical].fillna('NA')

In [48]:
df["industry"].mode()

0    retail
Name: industry, dtype: object

In [49]:
df["interaction_count"].corr(df["lead_score"])

0.009888182496913084

In [50]:
df["number_of_courses_viewed"].corr(df["lead_score"])

-0.004878998354681256

In [51]:
df["number_of_courses_viewed"].corr(df["interaction_count"])

-0.023565222882888044

In [52]:
df["annual_income"].corr(df["interaction_count"])

0.027036472404814348

In [53]:
from sklearn.metrics import mutual_info_score

In [54]:
for variable in ["industry", "location", "lead_source", "employment_status"]:
    print(variable, mutual_info_score(df["converted"], df[variable]))

industry 0.007267435279688886
location 0.0014269064526338665
lead_source 0.026573987738060995
employment_status 0.011069560968622674


In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X = df.drop("converted", axis=1)
y = df["converted"]

In [57]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [58]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [60]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [62]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [65]:
df_train.nunique()

lead_source                   6
industry                      8
number_of_courses_viewed     10
annual_income               765
employment_status             5
location                      8
interaction_count             9
lead_score                  101
dtype: int64

In [66]:
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,events,manufacturing,2,95543.0,unemployed,europe,3,0.78
1,referral,NA,1,54924.0,student,south_america,6,0.39
2,organic_search,healthcare,2,77352.0,unemployed,europe,2,0.22
3,paid_ads,other,2,34600.0,employed,south_america,2,0.31
4,paid_ads,education,0,43615.0,unemployed,south_america,2,0.01


In [67]:
from sklearn.feature_extraction import DictVectorizer

In [69]:
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]

In [70]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [72]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [73]:
y_pred = model.predict(X_val)

In [74]:
print(accuracy_score(y_pred, y_val))

0.6996587030716723
